# Data Warehouse usando Python

In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Funciones varias a utilizar

## Funcion para generar archivo .SQL a partir de un .CSV

In [200]:
import csv

def csv_a_sql(nombre_archivo_csv, nombre_archivo_sql, nombre_tabla):
    """
    Convierte un archivo CSV a un script SQL para crear una tabla e insertar datos.

    Args:
        nombre_archivo_csv: La ruta al archivo CSV de entrada.
        nombre_archivo_sql: La ruta al archivo SQL de salida.
        nombre_tabla: El nombre de la tabla SQL que se creará.
    """

    with open(nombre_archivo_csv, 'r', encoding='utf-8') as archivo_csv, open(nombre_archivo_sql, 'w', encoding='utf-8') as archivo_sql:
        lector_csv = csv.reader(archivo_csv)
        
        # Obtener la primera fila para los nombres de las columnas
        encabezado = next(lector_csv)

        # Crear la sentencia CREATE TABLE
        archivo_sql.write(f"CREATE TABLE {nombre_tabla} (\n")
        for nombre_columna in encabezado:
            # Limpiar y formatear el nombre de la columna para SQL
            nombre_columna = nombre_columna.replace(" ", "_").lower()  # Ejemplo: "Nombre Cliente" -> "nombre_cliente"
            archivo_sql.write(f"    {nombre_columna} TEXT,\n")  # Asume tipo TEXT, puedes ajustarlo
        archivo_sql.write(");\n\n")

        # Crear las sentencias INSERT INTO
        for fila in lector_csv:
            archivo_sql.write(f"INSERT INTO {nombre_tabla} (")
            archivo_sql.write(", ".join(f'{col.replace(" ", "_").lower()}' for col in encabezado))  # Columnas
            archivo_sql.write(") VALUES (")
            archivo_sql.write(", ".join(
                f"'{valor}'" if not valor.replace('.', '', 1).isdigit() else valor  # Verifica si es número
                for valor in fila
            ))
            archivo_sql.write(");\n")

# SISMOS

Cargar Dataset original de Sismos

In [201]:
archivo = "raw_data\\SSNMX_catalogo_19000101_20250213.csv"
data_sismos = pd.read_csv(archivo)
data_sismos.head()

C:\Users\Dell Latitude\AppData\Local\Temp\ipykernel_360\4089433804.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_sismos = pd.read_csv(archivo)


,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,1900-01-20,00:33:30,7.4,20.00,-105.00,33.0,"71 km al NOROESTE de AUTLAN DE NAVARRO, JAL",1900-01-20,06:33:30,revisado
1,1900-05-16,14:12:00,6.9,20.00,-105.00,33.0,"71 km al NOROESTE de AUTLAN DE NAVARRO, JAL",1900-05-16,20:12:00,revisado
2,1901-03-05,04:45:00,6.9,25.00,-110.00,33.0,"100 km al NORESTE de LA PAZ, BCS",1901-03-05,10:45:00,revisado
3,1901-12-08,20:17:00,7.0,26.00,-110.00,33.0,"83 km al OESTE de AHOME, SIN",1901-12-09,02:17:00,revisado
4,1902-01-16,17:19:00,7.0,17.62,-99.72,33.0,"21 km al OESTE de ZUMPANGO DEL RIO, GRO",1902-01-16,23:19:00,revisado


In [202]:
data_sismos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332391 entries, 0 to 332390
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Fecha                       332391 non-null  object 
 1   Hora                        332391 non-null  object 
 2   Magnitud                    332391 non-null  object 
 3   Latitud                     332391 non-null  float64
 4   Longitud                    332391 non-null  float64
 5   Profundidad                 332391 non-null  float64
 6   Referencia de localizacion  332391 non-null  object 
 7   Fecha UTC                   332391 non-null  object 
 8   Hora UTC                    332391 non-null  object 
 9   Estatus                     332391 non-null  object 
dtypes: float64(3), object(7)
memory usage: 25.4+ MB


In [203]:
data_sismos.describe()

,Latitud,Longitud,Profundidad
count,332391.000000,332391.00000,332391.000000
mean,18.033864,-99.37387,32.030894
std,4.229140,5.99640,36.170389
min,10.271000,-120.59500,0.000000
25%,15.931300,-101.76400,10.000000
50%,16.567300,-98.08920,16.100000
75%,18.154700,-94.91020,37.300000
max,38.092300,-85.54670,338.000000


Como podemos comprobar, las unicas columnas que se cargan como tipos de datos flotantes son las de Latitud, Longitud y Profundidad.
Por lo que podemos concluir que algunas otras como Magnitud que debería ser un valor numerico, puede contar con valores atipicos o que no pertenenezcan al mismo tipo de dato.

In [204]:
data_sismos.shape

(332391, 10)

In [205]:
data_sismos.columns

Index(['Fecha', 'Hora', 'Magnitud', 'Latitud', 'Longitud', 'Profundidad',
       'Referencia de localizacion', 'Fecha UTC', 'Hora UTC', 'Estatus'],
      dtype='object')

Verificar valores nulos

In [206]:
# valores nulos por columna
for columna in data_sismos.columns:
    print(f"{columna}: {data_sismos[columna].isnull().sum()}")


Fecha: 0
Hora: 0
Magnitud: 0
Latitud: 0
Longitud: 0
Profundidad: 0
Referencia de localizacion: 0
Fecha UTC: 0
Hora UTC: 0
Estatus: 0


Podríamos pensar que los datos para este punto están limpios, sin embargo analizando el archivo .CSV, podemos notar que los valores "nulos", están delimitados de forma diferente

In [207]:
data_sismos_clean = data_sismos.copy()

In [208]:
# Definir los valores que representan nulos
null_values = ["NA", "None", "-", "", "null", "no calculable"]

# Reemplazar esos valores con NaN
data_sismos_clean.replace(null_values, np.nan, inplace=True)

data_sismos_clean

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,1900-01-20,00:33:30,7.4,20.000,-105.000,33.0,"71 km al NOROESTE de AUTLAN DE NAVARRO, JAL",1900-01-20,06:33:30,revisado
1,1900-05-16,14:12:00,6.9,20.000,-105.000,33.0,"71 km al NOROESTE de AUTLAN DE NAVARRO, JAL",1900-05-16,20:12:00,revisado
2,1901-03-05,04:45:00,6.9,25.000,-110.000,33.0,"100 km al NORESTE de LA PAZ, BCS",1901-03-05,10:45:00,revisado
3,1901-12-08,20:17:00,7.0,26.000,-110.000,33.0,"83 km al OESTE de AHOME, SIN",1901-12-09,02:17:00,revisado
4,1902-01-16,17:19:00,7.0,17.620,-99.720,33.0,"21 km al OESTE de ZUMPANGO DEL RIO, GRO",1902-01-16,23:19:00,revisado
...,...,...,...,...,...,...,...,...,...,...
332386,2025-02-13,04:45:07,3.3,16.410,-98.442,35.0,"30 km al SUR de OMETEPEC, GRO",2025-02-13,10:45:07,verificado
332387,2025-02-13,04:54:04,3.5,17.657,-95.412,38.3,"37 km al SUR de J RODRIGUEZ CLARA, VER",2025-02-13,10:54:04,verificado
332388,2025-02-13,08:25:52,4.0,14.816,-98.058,16.0,"148 km al SUROESTE de RIO GRANDE, OAX",2025-02-13,14:25:52,verificado
332389,2025-02-13,08:42:33,4.0,15.782,-96.221,37.2,"9 km al OESTE de CRUCECITA, OAX",2025-02-13,14:42:33,verificado


Una vez remplazados, volvemos a buscar los valores nulos

In [209]:
# valores nulos por columna
for columna in data_sismos_clean.columns:
    print(f"{columna}: {data_sismos_clean[columna].isna().sum()}")

Fecha: 0
Hora: 0
Magnitud: 17649
Latitud: 0
Longitud: 0
Profundidad: 0
Referencia de localizacion: 0
Fecha UTC: 0
Hora UTC: 0
Estatus: 0


Eliminamos los valores nulos

In [210]:
# Eliminar filas con valores nulos
data_sismos_clean.dropna(inplace=True)

# Reiniciar los índices después de eliminar filas
data_sismos_clean.reset_index(drop=True, inplace=True)

data_sismos_clean

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,1900-01-20,00:33:30,7.4,20.000,-105.000,33.0,"71 km al NOROESTE de AUTLAN DE NAVARRO, JAL",1900-01-20,06:33:30,revisado
1,1900-05-16,14:12:00,6.9,20.000,-105.000,33.0,"71 km al NOROESTE de AUTLAN DE NAVARRO, JAL",1900-05-16,20:12:00,revisado
2,1901-03-05,04:45:00,6.9,25.000,-110.000,33.0,"100 km al NORESTE de LA PAZ, BCS",1901-03-05,10:45:00,revisado
3,1901-12-08,20:17:00,7.0,26.000,-110.000,33.0,"83 km al OESTE de AHOME, SIN",1901-12-09,02:17:00,revisado
4,1902-01-16,17:19:00,7.0,17.620,-99.720,33.0,"21 km al OESTE de ZUMPANGO DEL RIO, GRO",1902-01-16,23:19:00,revisado
...,...,...,...,...,...,...,...,...,...,...
314737,2025-02-13,04:45:07,3.3,16.410,-98.442,35.0,"30 km al SUR de OMETEPEC, GRO",2025-02-13,10:45:07,verificado
314738,2025-02-13,04:54:04,3.5,17.657,-95.412,38.3,"37 km al SUR de J RODRIGUEZ CLARA, VER",2025-02-13,10:54:04,verificado
314739,2025-02-13,08:25:52,4.0,14.816,-98.058,16.0,"148 km al SUROESTE de RIO GRANDE, OAX",2025-02-13,14:25:52,verificado
314740,2025-02-13,08:42:33,4.0,15.782,-96.221,37.2,"9 km al OESTE de CRUCECITA, OAX",2025-02-13,14:42:33,verificado


In [211]:
# valores nulos por columna
for columna in data_sismos_clean.columns:
    print(f"{columna}: {data_sismos_clean[columna].isna().sum()}")

Fecha: 0
Hora: 0
Magnitud: 0
Latitud: 0
Longitud: 0
Profundidad: 0
Referencia de localizacion: 0
Fecha UTC: 0
Hora UTC: 0
Estatus: 0


Una vez que el dataset está completamente limpio y que se mantienen las columnas necesarias, podemos decir que está listo

In [212]:
sismos = data_sismos_clean.copy()

### SISMOS LISTO

# POBLACIÓN

Cargar Dataset original de Población

In [213]:
archivo = "raw_data\\conjunto_de_datos_iter_00CSV20.csv"
data_poblacion = pd.read_csv(archivo)
data_poblacion.head()

C:\Users\Dell Latitude\AppData\Local\Temp\ipykernel_360\961270117.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data_poblacion = pd.read_csv(archivo)


,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,VPH_SINRTV,VPH_SINLTC,VPH_SINCINT,VPH_SINTIC,TAMLOC
0,0,Total nacional,0,Total nacional,0,Total nacional,NaN,NaN,NaN,126014024,...,30775898,18307193,15211306,6616141,4047100,1788552,3170894,15108204,852871,*
1,0,Total nacional,0,Total nacional,9998,Localidades de una vivienda,NaN,NaN,NaN,250354,...,47005,8385,18981,1732,1113,12775,14143,51293,7154,*
2,0,Total nacional,0,Total nacional,9999,Localidades de dos viviendas,NaN,NaN,NaN,147125,...,25581,5027,11306,971,708,8247,10065,29741,5283,*
3,1,Aguascalientes,0,Total de la entidad Aguascalientes,0,Total de la Entidad,NaN,NaN,NaN,1425607,...,359895,236003,174089,98724,70126,6021,15323,128996,1711,*
4,1,Aguascalientes,0,Total de la entidad Aguascalientes,9998,Localidades de una vivienda,NaN,NaN,NaN,3697,...,732,205,212,48,41,39,62,530,20,*


In [214]:
data_poblacion.shape

(195662, 286)

In [215]:
data_poblacion.describe()

,ENTIDAD,MUN,LOC,POBTOT,VIVTOT,TVIVHAB
count,195662.000000,195662.000000,195662.000000,1.956620e+05,1.956620e+05,1.956620e+05
mean,16.806743,59.699400,623.572221,2.582252e+03,9.000329e+02,7.218171e+02
std,8.940159,81.370934,1502.609425,2.931486e+05,1.019782e+05,8.195072e+04
min,0.000000,0.000000,0.000000,1.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000,12.000000,45.000000,4.000000e+00,2.000000e+00,1.000000e+00
50%,16.000000,33.000000,162.000000,1.600000e+01,7.000000e+00,4.000000e+00
75%,25.000000,73.000000,531.000000,1.470000e+02,5.400000e+01,3.800000e+01
max,32.000000,570.000000,9999.000000,1.260140e+08,4.390344e+07,3.523346e+07


In [216]:
data_poblacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 286 entries, ENTIDAD to TAMLOC
dtypes: int64(6), object(280)
memory usage: 426.9+ MB


In [217]:
data_poblacion.columns

Index(['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT',
       ...
       'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ',
       'VPH_SINRTV', 'VPH_SINLTC', 'VPH_SINCINT', 'VPH_SINTIC', 'TAMLOC'],
      dtype='object', length=286)

Verificar el filtrado de las columnas de intenteres del dataset original de sismos

In [218]:
data_poblacion.columns[:12]

Index(['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS'],
      dtype='object')

Generemos la copia del dataset filtrado

In [219]:
data_poblacion_sliced = data_poblacion.iloc[:, :12]

Verificar valores nulos en el dataset de poblacion

In [220]:
for column in data_poblacion_sliced.columns:
    print(data_poblacion_sliced[column].isna().sum(), "\t", column)

0 	 ENTIDAD
0 	 NOM_ENT
0 	 MUN
0 	 NOM_MUN
0 	 LOC
0 	 NOM_LOC
6230 	 LONGITUD
6230 	 LATITUD
6230 	 ALTITUD
0 	 POBTOT
0 	 POBFEM
0 	 POBMAS


In [221]:
null_counts = data_poblacion_sliced.apply(lambda col: (col == '*').sum())

print("Valores nulos por columna:")
print(null_counts)

Valores nulos por columna:
ENTIDAD         0
NOM_ENT         0
MUN             0
NOM_MUN         0
LOC             0
NOM_LOC         0
LONGITUD        0
LATITUD         0
ALTITUD         0
POBTOT          0
POBFEM      82011
POBMAS      82011
dtype: int64


In [222]:
# Reemplazar '*' con NaN
data_withNull = data_poblacion_sliced.replace('*', pd.NA)

In [223]:
for column in data_withNull.columns:
    print(data_withNull[column].isna().sum(), "\t", column)

0 	 ENTIDAD
0 	 NOM_ENT
0 	 MUN
0 	 NOM_MUN
0 	 LOC
0 	 NOM_LOC
6230 	 LONGITUD
6230 	 LATITUD
6230 	 ALTITUD
0 	 POBTOT
82011 	 POBFEM
82011 	 POBMAS


In [224]:
data_poblacion_sliced_clean = data_withNull.copy()

In [225]:
# Eliminar filas con valores nulos
data_poblacion_sliced_clean.dropna(inplace=True)

# Reiniciar los índices después de eliminar filas
data_poblacion_sliced_clean.reset_index(drop=True, inplace=True)

data_poblacion_sliced_clean.head()

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,POBFEM,POBMAS
0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102°17'45.768"" W","21°52'47.362"" N",1878.0,863893,444725,419168
1,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102°21'25.639"" W","21°53'01.522"" N",1861.0,41,17,24
2,1,Aguascalientes,1,Aguascalientes,106,Arellano,"102°16'26.238"" W","21°48'06.384"" N",1892.0,1169,556,613
3,1,Aguascalientes,1,Aguascalientes,112,Bajío los Vázquez,"102°07'29.341"" W","21°44'50.978"" N",1971.0,41,21,20
4,1,Aguascalientes,1,Aguascalientes,120,Buenavista de Peñuelas,"102°17'35.503"" W","21°43'08.930"" N",1871.0,1054,512,542


In [226]:
null_counts = data_poblacion_sliced_clean.isna().sum()

print("Valores nulos por columna:")
print(null_counts)

Valores nulos por columna:
ENTIDAD     0
NOM_ENT     0
MUN         0
NOM_MUN     0
LOC         0
NOM_LOC     0
LONGITUD    0
LATITUD     0
ALTITUD     0
POBTOT      0
POBFEM      0
POBMAS      0
dtype: int64


## Conversion de latitud y longitud a valores decimales

In [227]:
data_poblacion_sliced_clean_decimal = data_poblacion_sliced_clean.copy()

In [228]:
import re

# Función para convertir grados, minutos y segundos a decimal
def convertir_a_decimal(valor):
    # Eliminar espacios adicionales al inicio o al final
    valor = valor.strip()
    
    # Usar una expresión regular para extraer grados, minutos, segundos y dirección
    match = re.match(r"(\d+)°(\d+)'(\d+(?:\.\d+)?)\"?\s*([NSEW])", valor)
    if not match:
        raise ValueError(f"Formato inválido: {valor}")
    
    grados = float(match.group(1))
    minutos = float(match.group(2))
    segundos = float(match.group(3))
    direccion = match.group(4)
    
    # Convertir a decimal
    decimal = grados + (minutos / 60) + (segundos / 3600)
    
    # Si es sur (S) o oeste (W), el valor debe ser negativo
    if direccion in ["S", "W"]:
        decimal *= -1
    
    return decimal

In [229]:
# Aplicar la conversión a las columnas LATITUD y LONGITUD
data_poblacion_sliced_clean_decimal["LATITUD"] = data_poblacion_sliced_clean_decimal["LATITUD"].apply(convertir_a_decimal)
data_poblacion_sliced_clean_decimal["LONGITUD"] = data_poblacion_sliced_clean_decimal["LONGITUD"].apply(convertir_a_decimal)

data_poblacion_sliced_clean_decimal

,ENTIDAD,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,POBFEM,POBMAS
0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,-102.296047,21.879823,1878.0,863893,444725,419168
1,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,-102.357122,21.883756,1861.0,41,17,24
2,1,Aguascalientes,1,Aguascalientes,106,Arellano,-102.273955,21.801773,1892.0,1169,556,613
3,1,Aguascalientes,1,Aguascalientes,112,Bajío los Vázquez,-102.124817,21.747494,1971.0,41,21,20
4,1,Aguascalientes,1,Aguascalientes,120,Buenavista de Peñuelas,-102.293195,21.719147,1871.0,1054,512,542
...,...,...,...,...,...,...,...,...,...,...,...,...
108322,32,Zacatecas,58,Santa María de la Paz,30,Las Tablas,-103.261570,21.460688,2464.0,34,20,14
108323,32,Zacatecas,58,Santa María de la Paz,33,Los Horcones,-103.267078,21.498636,2152.0,12,4,8
108324,32,Zacatecas,58,Santa María de la Paz,35,Los Trigos (Mesa de los Trigos),-103.241053,21.497599,2499.0,89,43,46
108325,32,Zacatecas,58,Santa María de la Paz,37,Mesa Grande,-103.414701,21.553438,1944.0,165,88,77


Con los valores convertidos correctamente a decimal, el dataset está completamente limpio

In [230]:
poblacion = data_poblacion_sliced_clean.copy()

### POBLACIÓN LISTO

# ECONOMIA

Leer Dataset original economia

In [231]:
data_economia = pd.read_csv("raw_data\\economia_normalizada.csv")
data_economia.head()

,entidad,nombre_entidad,municipio,nombre_municipio,codigo,descripcion_actividad,clasificacion,produccion_bruta_total,insumos_utilizados,consumo_intermedio,valor_agregado,formacion_capital,activos_fijos_adquiridos
0,1,NaN,,NaN,1152,Servicios relacionados con la cra y explotacin...,Rama,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,,NaN,1152,Servicios relacionados con la cra y explotacin...,Rama,NaN,NaN,NaN,NaN,NaN,NaN
2,1,NaN,,NaN,624231,Servicios de emergencia comunitarios prestados...,Clase,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Aguascalientes,001,Aguascalientes,3253,"Fabricacin de fertilizantes, pesticidas y otro...",Rama,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NaN,,NaN,6243,Servicios de capacitacin para el trabajo para ...,Rama,NaN,NaN,NaN,-2.033,0.0,375.0


In [232]:
data_economia.describe()

,entidad,produccion_bruta_total,insumos_utilizados,consumo_intermedio,valor_agregado,formacion_capital,activos_fijos_adquiridos
count,1.805368e+06,8.470700e+05,8.470700e+05,8.470700e+05,1.277084e+06,1.277084e+06,1.277084e+06
mean,1.808536e+01,5.219418e+02,5.709560e+02,7.973581e+02,4.854818e+01,1.818187e+02,1.642086e+02
std,7.939655e+00,1.099325e+04,1.419730e+04,1.807074e+04,7.380273e+01,1.660482e+02,2.514614e+02
min,1.000000e+00,-6.640002e+03,-2.613200e+01,0.000000e+00,-5.558710e+02,0.000000e+00,0.000000e+00
25%,1.300000e+01,1.270000e+00,1.027000e+00,1.844000e+00,2.070400e+01,1.154930e+02,0.000000e+00
50%,1.700000e+01,6.210000e+00,5.216000e+00,9.165000e+00,3.580700e+01,1.743780e+02,0.000000e+00
75%,2.400000e+01,4.967400e+01,4.774700e+01,7.942450e+01,5.968700e+01,2.422770e+02,2.811415e+02
max,3.200000e+01,4.280287e+06,6.400326e+06,8.303585e+06,8.466667e+03,5.335391e+04,1.263154e+04


In [233]:
data_economia.shape

(1805368, 13)

In [234]:
data_economia.columns

Index(['entidad', 'nombre_entidad', 'municipio', 'nombre_municipio', 'codigo',
       'descripcion_actividad', 'clasificacion', 'produccion_bruta_total',
       'insumos_utilizados', 'consumo_intermedio', 'valor_agregado',
       'formacion_capital', 'activos_fijos_adquiridos'],
      dtype='object')

In [235]:
data_economia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1805368 entries, 0 to 1805367
Data columns (total 13 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   entidad                   int64  
 1   nombre_entidad            object 
 2   municipio                 object 
 3   nombre_municipio          object 
 4   codigo                    object 
 5   descripcion_actividad     object 
 6   clasificacion             object 
 7   produccion_bruta_total    float64
 8   insumos_utilizados        float64
 9   consumo_intermedio        float64
 10  valor_agregado            float64
 11  formacion_capital         float64
 12  activos_fijos_adquiridos  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 179.1+ MB


Contar valores nulos del dataset

In [236]:
# valores nulos
for columna in data_economia.columns:
    print(f"{columna}: {data_economia[columna].isnull().sum()}")

entidad: 0
nombre_entidad: 118049
municipio: 0
nombre_municipio: 118049
codigo: 0
descripcion_actividad: 6809
clasificacion: 6809
produccion_bruta_total: 958298
insumos_utilizados: 958298
consumo_intermedio: 958298
valor_agregado: 528284
formacion_capital: 528284
activos_fijos_adquiridos: 528284


Nos podemos dar cuenta de que existen una gran cantidad de valores nulos en columnas importantes como lo son la produccion bruta total, esto se puede deber a que el dataset contempla informacion no por estados sino por municipio
Al final lo que es necesario para nuestro analisis son los valores por estado, asi que haremos el experimento de borrar los registros con valores nulos

In [237]:
data_economia_clean = data_economia.copy()

In [238]:
# borrar los valores nulos  
data_economia_clean.dropna(inplace=True)
data_economia_clean.shape

(745970, 13)

Tras borrarlos obtenemos un dataset de 745 mil registros

In [239]:
data_economia_clean.head()

,entidad,nombre_entidad,municipio,nombre_municipio,codigo,descripcion_actividad,clasificacion,produccion_bruta_total,insumos_utilizados,consumo_intermedio,valor_agregado,formacion_capital,activos_fijos_adquiridos
8,28,Tamaulipas,032,Reynosa,337120,"Fabricacin de muebles, excepto cocinas integra...",Clase,153.033,73.438,153.225,37.382,348.510,634.058
17,28,Tamaulipas,032,Reynosa,33792,Fabricacin de persianas y cortineros,Subrama,913.806,450.292,913.706,0.668,288.928,724.590
18,28,Tamaulipas,032,Reynosa,33792,Fabricacin de persianas y cortineros,Subrama,913.806,450.292,913.706,0.668,288.928,724.590
19,28,Tamaulipas,032,Reynosa,337920,Fabricacin de persianas y cortineros,Clase,913.806,450.292,913.706,0.668,288.928,724.590
20,28,Tamaulipas,032,Reynosa,337920,Fabricacin de persianas y cortineros,Clase,913.806,450.292,913.706,0.668,288.928,724.590


In [240]:
data_economia_clean.describe()

,entidad,produccion_bruta_total,insumos_utilizados,consumo_intermedio,valor_agregado,formacion_capital,activos_fijos_adquiridos
count,745970.000000,745970.000000,7.459700e+05,7.459700e+05,745970.000000,745970.000000,745970.000000
mean,17.839573,206.327969,2.331823e+02,3.231380e+02,47.181789,185.246481,158.093111
std,7.882226,3165.058729,4.899387e+03,5.595603e+03,66.538265,155.596542,223.731469
min,1.000000,-6640.002000,-2.613200e+01,0.000000e+00,-555.871000,0.000000,0.000000
25%,12.000000,1.072000,8.670000e-01,1.555000e+00,22.455000,129.677000,0.000000
50%,16.000000,4.418000,3.734000e+00,6.576000e+00,36.237000,176.382000,0.000000
75%,24.000000,27.328000,2.680200e+01,4.447800e+01,58.026000,236.607000,271.685500
max,32.000000,475017.229000,1.300367e+06,1.342808e+06,8466.667000,53353.911000,11487.430000


Observamos un dataset el cual es muy trabajable, sin valores nulos y ordenado, en este dataset podemos tener los datos por municipio, pero por fines practicos, buscaremos obtener un dataset que separe unicamente por estado

In [241]:
data_economia_clean["nombre_entidad"].unique()

array(['Tamaulipas', 'Campeche', 'Chihuahua', 'Chiapas', 'Puebla',
       'San Luis Potos', 'Tlaxcala', 'Baja California Sur', 'Colima',
       'Guerrero', 'Ciudad de Mxico', 'Guanajuato', 'Morelos',
       'Zacatecas', 'Jalisco', 'Mxico', 'Aguascalientes', 'Nuevo Len',
       'Oaxaca', 'Coahuila de Zaragoza', 'Michoacn de Ocampo', 'Sonora',
       'Durango', 'Baja California', 'Hidalgo', 'Quertaro de Arteaga',
       'Nayarit', 'Sinaloa', 'Tabasco', 'Quintana Roo',
       'Veracruz de Ignacio de la Llave', 'Yucatn'], dtype=object)

In [242]:
# Seleccionar solo las columnas relacionadas con los estados y las numéricas
columnas_estado = [
    "entidad", 
    "nombre_entidad", 
    "produccion_bruta_total", 
    "insumos_utilizados", 
    "consumo_intermedio", 
    "valor_agregado", 
    "formacion_capital", 
    "activos_fijos_adquiridos"
]

# Filtrar el dataset para mantener solo las columnas seleccionadas
data_economia_clean_estado = data_economia_clean[columnas_estado]

# Agrupar por estado (nombre_entidad) y calcular la media para las columnas numéricas
data_economia_clean_estado = data_economia_clean_estado.groupby("nombre_entidad").mean().reset_index()

# Mostrar el resultado
data_economia_clean_estado

,nombre_entidad,entidad,produccion_bruta_total,insumos_utilizados,consumo_intermedio,valor_agregado,formacion_capital,activos_fijos_adquiridos
0,Aguascalientes,1.0,415.688970,406.180314,565.485103,39.171273,212.967537,207.990443
1,Baja California,2.0,635.289316,619.101969,952.796500,44.332165,283.455966,376.115924
2,Baja California Sur,3.0,212.644686,259.987752,365.475133,40.776352,243.462586,274.606066
3,Campeche,4.0,96.423859,110.162071,163.581780,45.907354,186.207157,167.538823
4,Chiapas,7.0,31.554876,46.001539,62.317679,48.333857,140.981941,90.105823
5,Chihuahua,8.0,263.100868,264.861724,404.821288,42.398126,218.362213,220.233976
6,Ciudad de Mxico,9.0,1316.185878,1848.554282,2456.910854,46.411031,278.451751,357.767460
7,Coahuila de Zaragoza,5.0,518.970620,448.933074,642.162673,45.672469,223.652439,242.864775
8,Colima,6.0,101.932057,140.681912,194.297141,41.299012,212.835520,209.796288
9,Durango,10.0,129.170771,143.939978,202.822768,52.440776,186.527012,146.204069


Con lo anterior, nos quedamos con *2* datasets, uno que cuenta con la informacion total y especifica por municipios y otro que cuenta unicamente con la informacion de los estados

In [243]:
economia_municipios = data_economia_clean.copy()
economia_estados = data_economia_clean_estado.copy()

ECONOMIA LISTO

# Procedimiento para pasarlos a una base de datos

Primero deberán ser convertidos a archivos .CSV de los cuales facilmente se pueden obtener archivos .SQL para que los mismos sean usados en cualquier sistema gestor de bases de datos

Generar archivs .CSV de todos los datasets trabajados: 
* sismos
* poblacion
* economia_municipio
* economia_estados

In [244]:
sismos.to_csv("raw_data_limpio/sismos.csv", index=False)
poblacion.to_csv("raw_data_limpio/poblacion.csv", index=False)
economia_municipios.to_csv("raw_data_limpio/economia_municipios.csv", index=False)
economia_estados.to_csv("raw_data_limpio/economia_estados.csv", index=False)

# Procedimiento para convertir los archivos .CSV generados a archivos .SQL

In [245]:
for archivo_csv in ["raw_data_limpio/sismos.csv", "raw_data_limpio/poblacion.csv", "raw_data_limpio/economia_municipios.csv", "raw_data_limpio/economia_estados.csv"]:
    archivo_sql = archivo_csv.split('/')[-1].replace(".csv", ".sql")
    nombre_tabla = archivo_csv.split("/")[-1].replace(".csv", "")
    csv_a_sql(archivo_csv, "SQLs/" + archivo_sql, nombre_tabla)